In [212]:
import sympy as sp
from sympy.abc import t
from sympy.physics.quantum import TensorProduct
import typing as tp
i = sp.I

In [213]:
class Qubits:
    ZERO = sp.Matrix([1,0])
    ONE = sp.Matrix([0,1])

In [214]:
class Q:
    X = sp.Matrix([
        [0, 1],
        [1, 0]
    ])
    Y = sp.Matrix([
        [0, -i],
        [i, 0]
    ])
    Z = sp.Matrix([
        [1, 0],
        [0, -1]
    ])
    H = sp.Matrix([
        [1, 1],
        [1, -1]
    ])/sp.sqrt(2)
    T = sp.Matrix([
        [1, 0],
        [0, sp.exp(i*sp.pi/4)]
    ])
    CX = sp.Matrix([
        [sp.eye(2), sp.zeros(2)],
        [sp.zeros(2), X]
    ])
    CU = lambda U: sp.Matrix([
        [sp.eye(2), sp.zeros(2)],
        [sp.zeros(2), U]
    ])
    CCNOT = sp.Matrix([
        [sp.eye(4), sp.zeros(4)],
        [sp.zeros(4), CX]
    ])
    SWAP = sp.Matrix([
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
    ])
    P = lambda phi: sp.Matrix([
        [1, 0],
        [0, sp.exp(i*phi)]
    ])
    XC = SWAP@CX@SWAP
    I = sp.eye(2)


In [215]:
from functools import reduce


class QAlgo:
    def __init__(self, str: tp.List[tp.LiteralString]):
        assert len(str) <= 2
 
        str = [s.replace("-", "I") for s in str]

        self.str = str

    def execGate(self, C: tp.List[tp.LiteralString], state: sp.Matrix):
        control = getattr(Q, "".join(C), None)
        matrix = reduce(TensorProduct, [getattr(Q, c) for c in C], sp.Matrix([1])) if control == None else control
        return matrix @ state

    def process(self, primaryState: sp.Matrix):
        state = reduce(TensorProduct, primaryState, sp.Matrix([1]))
        for gates in zip(*self.str):
            state = self.execGate(gates, state)
        return state
    
    def matrixForm(self):
        matrix = sp.eye(len(self.str)**2)
        for gates in zip(*self.str):
            matrix = self.execGate(gates, matrix)
        return matrix

In [219]:
algo = QAlgo(["HCY-HX", "HXZHHC"])
q0 = Qubits.ZERO
q1 = Qubits.ZERO
display(algo.process([q0,q1]))
U=algo.matrixForm()
U

Matrix([
[           0],
[ sqrt(2)*I/2],
[-sqrt(2)*I/2],
[           0]])

Matrix([
[           0, sqrt(2)*I/2,  sqrt(2)*I/2,            0],
[ sqrt(2)*I/2,           0,            0, -sqrt(2)*I/2],
[-sqrt(2)*I/2,           0,            0, -sqrt(2)*I/2],
[           0, sqrt(2)*I/2, -sqrt(2)*I/2,            0]])

In [217]:
U@sp.Matrix([1,0,0,0])

Matrix([
[           0],
[ sqrt(2)*I/2],
[-sqrt(2)*I/2],
[           0]])

In [246]:
# Deutsch-Jozsa: 
deutsch = QAlgo(["HCH", "HX-"]) # f(x) = !x
# deutsch = QAlgo(["H-H", "HX-"]) # f(x) = 1
# deutsch = QAlgo(["H-H", "H--"]) # f(x) = 0
deutsch.process([Qubits.ZERO, Qubits.ONE])

Matrix([
[         0],
[         0],
[ sqrt(2)/2],
[-sqrt(2)/2]])